# SentiMind, Sentence Sentiment Clasfier
### Input and Output
The input is a sentence or sentences. The output is a list of 0-26 represent one or more sentiment including admiration, gratitude, surprise, or neutural.

### Architecture
the model or architecture we plan is 
1. Tokenizing the input
2. use one of the pretrained encoder transformer from hugging face to process the embedding, make it context aware to increase accuracy.
3. pass the context aware embedding to multiple layer of CNN, maybe 6 layers, each of a configuration of VGG level. 
4. connect the output of CNN to flattened and connect to FNN to make final classfication.
5. softmax the result, include all class with prob greater than a threshold. For example, if a class is more than 0.3, include as output. anger 0.4, remorese 0.35, rest 0.25, output [anger, remorses].

##### Step 1: Tokenizing the input

In [41]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import torch.nn.functional as F

# for plotting
import matplotlib.pyplot as plt

# for algebric computations
import numpy as np
from pprint import pprint

In [39]:
id_to_class = {
    0: 'Admiration',
    1: 'Amusement',
    2: 'Anger',
    3: 'Annoyance',
    4: 'Approval',
    5: 'Caring',
    6: 'Confusion',
    7: 'Curiosity',
    8: 'Desire',
    9: 'Disappointment',
    10: 'Disapproval',
    11: 'Disgust',
    12: 'Embarrassment',
    13: 'Excitement',
    14: 'Fear',
    15: 'Gratitude',
    16: 'Grief',
    17: 'Joy',
    18: 'Love',
    19: 'Nervousness',
    20: 'Optimism',
    21: 'Pride',
    22: 'Realization',
    23: 'Relief',
    24: 'Remorse',
    25: 'Sadness',
    26: 'Surprise'
    }

pprint(id_to_class)

{0: 'Admiration',
 1: 'Amusement',
 2: 'Anger',
 3: 'Annoyance',
 4: 'Approval',
 5: 'Caring',
 6: 'Confusion',
 7: 'Curiosity',
 8: 'Desire',
 9: 'Disappointment',
 10: 'Disapproval',
 11: 'Disgust',
 12: 'Embarrassment',
 13: 'Excitement',
 14: 'Fear',
 15: 'Gratitude',
 16: 'Grief',
 17: 'Joy',
 18: 'Love',
 19: 'Nervousness',
 20: 'Optimism',
 21: 'Pride',
 22: 'Realization',
 23: 'Relief',
 24: 'Remorse',
 25: 'Sadness',
 26: 'Surprise'}


In [29]:
# Choose a model
model_name = "distilbert-base-uncased"  # Replace with "albert-base-v2", "distilroberta-base", or "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
embedding_size = model.config.hidden_size


In [24]:
# Example input text
texts = ["I love this product! something, no matter how long as long as this is ", "The experience was terrible that I don't want to come any more and the number is depend on the longest sequence"]

# Tokenize input
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
decoded_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in inputs.input_ids]
for i, text in enumerate(decoded_texts):
    print(f"Decoded Text {i+1}: {text}")

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)

# Output: last_hidden_state (embeddings of shape [batch_size, seq_len, hidden_dim])
last_hidden_state = outputs.last_hidden_state

print(last_hidden_state[0][0])
print(f"Last Hidden State Shape: {last_hidden_state.shape}")

Decoded Text 1: i love this product! something, no matter how long as long as this is
Decoded Text 2: the experience was terrible that i don ' t want to come any more and the number is depend on the longest sequence
tensor([ 1.5044e-01,  1.8223e-02,  1.3812e-01, -1.4606e-01,  1.0705e-02,
        -2.3983e-01,  1.1390e-01,  6.2418e-01, -1.5608e-01, -2.2146e-01,
         9.4245e-03, -2.3805e-01,  1.2921e-02,  6.4960e-01,  7.2195e-02,
         5.0208e-02, -9.9061e-02,  3.9943e-01,  1.3665e-01,  8.4229e-03,
        -1.6087e-01, -2.5076e-01,  4.9469e-02,  1.1748e-01, -1.3496e-03,
        -1.1265e-01, -4.6018e-02, -1.4053e-01,  1.2362e-01, -4.3372e-03,
         5.1253e-02,  4.6455e-02, -2.1266e-01, -2.0892e-01,  2.5433e-03,
        -1.5094e-01,  2.6488e-02, -1.6228e-01, -5.6696e-02, -3.9978e-02,
        -1.4650e-01,  1.7114e-02,  1.8113e-01, -1.6018e-01, -2.8664e-01,
        -2.7828e-01, -2.4839e+00, -2.5722e-02, -1.5833e-01, -7.7120e-02,
         2.9434e-01, -2.7885e-02, -1.7000e-02,  1.3479

### Step 3: pass the context aware embedding to multiple layer of CNN

In [43]:
class CNN_Model(torch.nn.Module):
    def __init__(self, embedding_size):
        """
        k1 k2: conv layer size, conv layer size: k1 x embedding_size
        n1 n2: channel #
        """
        super().__init__()
        k1 = 2
        k2 = 4
        channel_size1 = 10
        channel_size2 = 20

        kernal_size1 = (k1, embedding_size)
        kernal_size2 = (k2, embedding_size)

        self.conv1 = torch.nn.Conv2d(1, channel_size1, kernal_size1, bias=False)
        self.conv2 = torch.nn.Conv2d(1, channel_size2, kernal_size2, bias=False)

        self.output_model = torch.nn.Linear(channel_size1 + channel_size2 , 26)


    def forward(self, embedded_text):
        # embedded_text: [2, 7, 768] batch 2, all sentences max length 8, embedding size 768
        sentence_emb = embedded_text.unsqueeze(1)  # for conv2d()

        conv1_out = self.conv1(sentence_emb)
        conv2_out = self.conv2(sentence_emb)

        conv1_out = torch.nn.functional.relu(conv1_out)
        conv2_out = torch.nn.functional.relu(conv2_out)

        conv1_out = conv1_out.squeeze(3)
        conv2_out = conv2_out.squeeze(3)

        # maxpool
        conv1_out = torch.nn.functional.max_pool1d(conv1_out, conv1_out.size(2)).squeeze(2)  # [batch_size, n1]
        conv2_out = torch.nn.functional.max_pool1d(conv2_out, conv2_out.size(2)).squeeze(2)  # [batch_size, n2]


        after_maxpool = torch.cat((conv1_out, conv2_out), 1)
        # return torch.sigmoid(self.output_model(after_maxpool))  # apply sigmoid
        distribution = self.output_model(after_maxpool)
        return F.softmax(distribution, dim=1)

In [44]:
CNN = CNN_Model(embedding_size)

output = CNN(last_hidden_state)

In [45]:
print(output)

tensor([[0.0289, 0.0212, 0.0436, 0.0337, 0.0380, 0.0396, 0.0384, 0.0260, 0.0361,
         0.0290, 0.0407, 0.0430, 0.0395, 0.0414, 0.0395, 0.0469, 0.0516, 0.0552,
         0.0408, 0.0379, 0.0384, 0.0574, 0.0349, 0.0452, 0.0269, 0.0260],
        [0.0288, 0.0225, 0.0462, 0.0371, 0.0390, 0.0378, 0.0387, 0.0285, 0.0368,
         0.0286, 0.0399, 0.0433, 0.0385, 0.0428, 0.0434, 0.0492, 0.0454, 0.0501,
         0.0402, 0.0355, 0.0389, 0.0548, 0.0335, 0.0437, 0.0295, 0.0273]],
       grad_fn=<SoftmaxBackward0>)
